In [1]:
import pandas as pd
import glob

files = glob.glob('D:\lc data\*.csv') 

pd.set_option('display.max_columns', None)

In [2]:
# For reading columns that are dates
lc_date_parser = lambda date: pd.to_datetime(date, format='%b-%Y', errors='coerce')
date_column_names = ['issue_d', 'earliest_cr_line', 'last_pymnt_d', 'last_credit_pull_d', 'next_pymnt_d' ]

# Read _all_ the csv's
loans = pd.concat( pd.read_csv( f , skiprows=1, header=0, engine='python', parse_dates=date_column_names, date_parser=lc_date_parser, skipfooter=2) for f in files )

# Drop some columns to save space
columns_to_drop = ['url', 'policy_code', 'application_type']
loans.drop( columns_to_drop, axis=1, inplace=True)

### String manipulations
Probably don't need regex for some columns here

In [3]:
loans['int_rate']		= loans['int_rate'].replace(to_replace='%', value='', regex=True).astype('float16')
loans['revol_util'] 	= loans['revol_util'].replace(to_replace='%', value='', regex=True).astype('float16')
loans['emp_length'] 	= loans['emp_length'].replace( '[^\d]+', '', regex=True).replace('', value=0).astype('int8')
loans['zip_code'] 	= loans['zip_code'].str.replace('xx', '00').astype('int32')
loans['desc'] = loans['desc'].str.replace(r"Borrower added on .*? \>", "").str.strip()

### Type conversions
To reduce memory usage

In [4]:

# Type conversions for columns I know should be integers or smaller floats
loans['id'] 					= loans['id'].astype('int32')
loans['member_id'] 				= loans['member_id'].astype('int32')
loans['loan_amnt'] 				= loans['loan_amnt'].astype('int32')
loans['funded_amnt'] 				= loans['funded_amnt'].astype('int32')
loans['funded_amnt_inv'] 			= loans['funded_amnt_inv'].astype('int32')
loans['annual_inc'] 				= loans['annual_inc'].astype('int32')

loans['delinq_2yrs'] 				= loans['delinq_2yrs'].astype('int8')
loans['fico_range_low'] 			= loans['fico_range_low'].astype('int16')
loans['fico_range_high'] 			= loans['fico_range_high'].astype('int16')


loans['inq_last_6mths'] 			= loans['inq_last_6mths'].astype('int8')
loans['open_acc'] 				= loans['open_acc'].astype('int8')
loans['revol_bal'] 				= loans['revol_bal'].astype('int32')
loans['total_acc'] 				= loans['total_acc'].astype('int8')

loans['last_fico_range_low'] 		= loans['fico_range_low'].astype('int16')
loans['last_fico_range_high'] 		= loans['fico_range_high'].astype('int16')
loans['collections_12_mths_ex_med'] 	= loans['collections_12_mths_ex_med'].astype('float16')
loans['acc_now_delinq']			= loans['acc_now_delinq'].astype('int8')

loans['int_rate'] 				= loans['int_rate'].astype('float16')

# Anything else can be float32 instead of float64
for column in loans.select_dtypes(include=['float64']):
    loans[column] = loans[column].astype('float32')	




### Flag loans as good or bad
Anything that isn't fully paid or current is considered bad

In [5]:
loans['bad'] =  (~loans.loan_status.isin( ['Fully Paid', 'Current'] )).astype('int8')

### Finish up and save to pickle
Using pickle just so it's easier to load into other scripts for analytics

In [ ]:
loans.set_index( 'id', inplace=True )

# To pickle for use in other scripts
loans.to_pickle('C:\lc data\lc.pickle');

In [10]:
loans.head(5)

,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,desc,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,fico_range_low,fico_range_high,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,next_pymnt_d,last_credit_pull_d,last_fico_range_high,last_fico_range_low,collections_12_mths_ex_med,mths_since_last_major_derog,annual_inc_joint,dti_joint,verification_status_joint,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_acc_6m,open_il_6m,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m,bad
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1077501,1296599,5000,5000,4975,36 months,10.648438,162.869995,B,B2,NaN,10,RENT,24000,Verified,2011-12-01,Fully Paid,n,I need to upgrade my business technologies.<br>,credit_card,Computer,86000,AZ,27.650000,0,1985-01-01,735,739,1,NaN,NaN,3,0,13648,83.687500,9,f,0.000000,0.000000,5861.071289,5831.779785,5000.000000,861.070007,0.000000,0.000000,0.00,2015-01-01,171.619995,NaT,2016-01-01,739,735,0.0,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
1077430,1314167,2500,2500,2500,60 months,15.273438,59.830002,C,C4,Ryder,1,RENT,30000,Source Verified,2011-12-01,Charged Off,n,I plan to use this money to finance the motorc...,car,bike,30900,GA,1.000000,0,1999-04-01,740,744,5,NaN,NaN,3,0,1687,9.398438,4,f,0.000000,0.000000,1008.710022,1008.710022,456.459991,435.170013,0.000000,117.080002,1.11,2013-04-01,119.660004,NaT,2013-09-01,744,740,0.0,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
1077175,1313524,2400,2400,2400,36 months,15.960938,84.330002,C,C5,NaN,10,RENT,12252,Not Verified,2011-12-01,Fully Paid,n,NaN,small_business,real estate business,60600,IL,8.720000,0,2001-11-01,735,739,2,NaN,NaN,2,0,2956,98.500000,10,f,0.000000,0.000000,3003.653564,3003.649902,2400.000000,603.650024,0.000000,0.000000,0.00,2014-06-01,649.909973,NaT,2016-01-01,739,735,0.0,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
1076863,1277178,10000,10000,10000,36 months,13.492188,339.309998,C,C1,AIR RESOURCES BOARD,10,RENT,49200,Source Verified,2011-12-01,Fully Paid,n,"to pay for property tax (borrow from friend, n...",other,personel,91700,CA,20.000000,0,1996-02-01,690,694,1,35.0,NaN,10,0,5598,21.000000,37,f,0.000000,0.000000,12226.301758,12226.299805,10000.000000,2209.330078,16.969999,0.000000,0.00,2015-01-01,357.480011,NaT,2015-01-01,694,690,0.0,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
1075358,1311748,3000,3000,3000,60 months,12.687500,67.790001,B,B5,University Medical Group,1,RENT,80000,Source Verified,2011-12-01,Current,n,I plan on combining three large interest bills...,other,Personal,97200,OR,17.940001,0,1996-01-01,695,699,0,38.0,NaN,15,0,27783,53.906250,38,f,766.900024,766.900024,3242.169922,3242.169922,2233.100098,1009.070007,0.000000,0.000000,0.00,2016-01-01,67.790001,2016-02-01,2016-01-01,699,695,0.0,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0


In [11]:
loans.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 884633 entries, 1077501 to 36271262
Data columns (total 75 columns):
member_id                      884633 non-null int32
loan_amnt                      884633 non-null int32
funded_amnt                    884633 non-null int32
funded_amnt_inv                884633 non-null int32
term                           884633 non-null object
int_rate                       884633 non-null float16
installment                    884633 non-null float32
grade                          884633 non-null object
sub_grade                      884633 non-null object
emp_title                      833334 non-null object
emp_length                     884633 non-null int8
home_ownership                 884633 non-null object
annual_inc                     884633 non-null int32
verification_status            884633 non-null object
issue_d                        884633 non-null datetime64[ns]
loan_status                    884633 non-null object
pymnt_plan    